<a href="https://colab.research.google.com/github/icaroc/blender-render-colab/blob/main/blender_render.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Inicio
Especifique abaixo a versão do Blender que quer renderizar e o caminho no Drive para os arquivos.

###Info
Se você precisa de mais informações nos parametros e etc. Veja aqui: [Blender CLI Wiki](https://docs.blender.org/manual/en/latest/advanced/command_line/arguments.html)

In [ ]:
#@title Configurações
#@markdown Por favor, configure as informações:

blender_version = 'blender2.91.2' #@param ["blender2.80", "blender2.81", "blender2.82", "blender2.83", "blender2.90.1", "blender2.91.2"] {allow-input: false}
caminho_arquivo_blend = '/Colab Notebooks/'  #@param {type: "string"}
caminho_saida_render = '/Colab Notebooks/'  #@param {type: "string"}
gpu_enabled = True #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}
#@markdown ---


# Baixando e Configurando o Blender

Rode todos esses blocos para instalação e correções.

In [ ]:
if blender_version == "blender2.80":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.80/blender-2.80-linux-glibc217-x86_64.tar.bz2"
elif blender_version == "blender2.81":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.81/blender-2.81-linux-glibc217-x86_64.tar.bz2"
elif blender_version == "blender2.82":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.82/blender-2.82-linux64.tar.xz"
elif blender_version == "blender2.83":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.83/blender-2.83.0-linux64.tar.xz"
elif blender_version == "blender2.90.1":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.90/blender-2.90.1-linux64.tar.xz"
elif blender_version == "blender2.91.2":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.91/blender-2.91.2-linux64.tar.xz"

In [ ]:
!mkdir $blender_version
if blender_version == "blender2.81" or "blender2.80":
    !wget -O '{blender_version}.tar.bz2' -nc $download_path
    !tar -xf '{blender_version}.tar.bz2' -C ./$blender_version --strip-components=1
else:
    !wget -O '{blender_version}.tar.xz' -nc $download_path
    !tar xf '{blender_version}.tar.xz' -C ./$blender_version --strip-components=1


In [ ]:
import os

os.environ["LD_PRELOAD"] = ""

!apt update
!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4
os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"

!echo $LD_PRELOAD

In [ ]:
!apt install libboost-all-dev
!apt install libgl1-mesa-dev
!apt install libglu1-mesa libsm-dev

Se você trocar o reder de GPU-CPU, rode essa célula novamente.

In [ ]:
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "# Attempt to set GPU device types if available\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "#for scene in bpy.data.scenes:\n"+\
    "#    scene.render.tile_x = 64\n"+\
    "#    scene.render.tile_y = 64\n"+\
    "# Enable all CPU and GPU devices\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

# Renderizando

## Montando o Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Use essa célula para renderizar apenas 1 frame.
Especifique o frame que vai ser renderizado no parametro `-f *frame_number*`

In [ ]:
!sudo ./$blender_version/blender -P 'setgpu.py' -b '/gdrive/My Drive/{caminho_arquivo_blend}' -P 'setgpu.py' -o '/gdrive/My Drive/{caminho_saida_render}' -f 1

## Use essa célula para renderizar uma animação.

Parametro -s especifica o frame inicial.
eg: `-s 10`

Parametro -e especifica o frame final.
eg: `-e 20`

**A ORDEM É IMPORTANTE. AMBOS `-s` E `-e` PRECISAM SER COLOCADOS ANTES DO `-a`**

In [ ]:
!sudo ./$blender_version/blender -P './setgpu.py'  -b '/gdrive/My Drive/{caminho_arquivo_blend}' -E CYCLES -o '/gdrive/My Drive/{caminho_saida_render}' -a